In [7]:
!pip install xlwings pandas appscript psutil -q

In [8]:
import xlwings as xw
import pandas as pd
from time import sleep

# --- CONFIGURATION ---
excel_file = "/Users/luizbeethoven/Documents/EXCELS/dgeg-ect-2024.xlsx"
sheet_name = "DGEG"

municipio_cell = "E11"   # Município dropdown
data_range = "A15:E21"   # table values only (categories + 4 numeric cols)

output_csv = "/Users/luizbeethoven/Desktop/municipios_output.csv"

# --- MUNICIPALITIES LIST ---
municipios = [
    "Águeda","Albergaria-a-Velha","Anadia","Arouca","Aveiro","Castelo de Paiva",
    "Espinho","Estarreja","Santa Maria da Feira","Ílhavo","Mealhada","Murtosa",
    "Oliveira de Azeméis","Oliveira do Bairro","Ovar","São João da Madeira",
    "Sever do Vouga","Vagos","Vale de Cambra",
    "Aljustrel","Almodôvar","Alvito","Barrancos","Beja","Castro Verde","Cuba",
    "Ferreira do Alentejo","Mértola","Moura","Odemira","Ourique","Serpa","Vidigueira",
    "Amares","Barcelos","Braga","Cabeceiras de Basto","Celorico de Basto",
    "Esposende","Fafe","Guimarães","Póvoa de Lanhoso","Terras de Bouro",
    "Vieira do Minho","Vila Nova de Famalicão","Vila Verde","Vizela",
    "Alfândega da Fé","Bragança","Carrazeda de Ansiães","Freixo de Espada à Cinta",
    "Macedo de Cavaleiros","Miranda do Douro","Mirandela","Mogadouro",
    "Torre de Moncorvo","Vila flor","Vimioso","Vinhais",
    "Belmonte","Castelo Branco","Covilhã","Fundão","Idanha-a-Nova","Oleiros",
    "Penamacor","Proença-a-Nova","Sertã","Vila de Rei","Vila Velha de Ródão",
    "Arganil","Cantanhede","Coimbra","Condeixa-a-Nova","Figueira da Foz","Góis",
    "Lousã","Mira","Miranda do Corvo","Montemor-o-Velho","Oliveira do Hospital",
    "Pampilhosa da Serra","Penacova","Penela","Soure","Tábua","Vila Nova de Poiares",
    "Alandroal","Arraiolos","Borba","Estremoz","Évora","Montemor-o-Novo","Mora",
    "Mourão","Portel","Redondo","Reguengos de Monsaraz","Vendas Novas",
    "Viana do Alentejo","Vila Viçosa",
    "Albufeira","Alcoutim","Aljezur","Castro Marim","Faro","Lagoa (Algarve)","Lagos",
    "Loulé","Monchique","Olhão","Portimão","São Brás de Alportel","Silves","Tavira",
    "Vila do Bispo","Vila Real de Santo António",
    "Aguiar da Beira","Almeida","Celorico da Beira","Figueira de Castelo Rodrigo",
    "Fornos de Algodres","Gouveia","Guarda","Manteigas","Meda","Pinhel","Sabugal",
    "Seia","Trancoso","Vila Nova de Foz Côa",
    "Alcobaça","Alvaiázere","Ansião","Batalha","Bombarral","Caldas da Rainha",
    "Castanheira de Pêra","Figueiró dos Vinhos","Leiria","Marinha Grande","Nazaré",
    "Óbidos","Pedrógão Grande","Peniche","Pombal","Porto de Mós",
    "Alenquer","Arruda dos Vinhos","Azambuja","Cadaval","Cascais","Lisboa","Loures",
    "Lourinhã","Mafra","Oeiras","Sintra","Sobral de Monte Agraço","Torres Vedras",
    "Vila Franca de Xira","Amadora","Odivelas",
    "Alter do Chão","Arronches","Avis","Campo Maior","Castelo de Vide","Crato",
    "Elvas","Fronteira","Gavião","Marvão","Monforte","Nisa","Ponte de Sôr",
    "Portalegre","Sousel",
    "Amarante","Baião","Felgueiras","Gondomar","Lousada","Maia","Marco de Canaveses",
    "Matosinhos","Paços de Ferreira","Paredes","Penafiel","Porto","Póvoa de Varzim",
    "Santo Tirso","Valongo","Vila do Conde","Vila Nova de Gaia","Trofa",
    "Abrantes","Alcanena","Almeirim","Alpiarça","Benavente","Cartaxo","Chamusca",
    "Constância","Coruche","Entroncamento","Ferreira do Zêzere","Golegã","Mação",
    "Rio Maior","Salvaterra de Magos","Santarém","Sardoal","Tomar","Torres Novas",
    "Vila Nova da Barquinha","Ourém",
    "Alcácer do Sal","Alcochete","Almada","Barreiro","Grândola","Moita","Montijo",
    "Palmela","Santiago do Cacém","Seixal","Sesimbra","Setúbal","Sines",
    "Arcos de Valdevez","Caminha","Melgaço","Monção","Paredes de Coura",
    "Ponte da Barca","Ponte de Lima","Valença","Viana do Castelo","Vila Nova de Cerveira",
    "Alijó","Boticas","Chaves","Mesão Frio","Mondim de Basto","Montalegre","Murça",
    "Peso da Régua","Ribeira de Pena","Sabrosa","Santa Marta de Penaguião",
    "Valpaços","Vila Pouca de Aguiar","Vila Real",
    "Armamar","Carregal do Sal","Castro Daire","Cinfães","Lamego","Mangualde",
    "Moimenta da Beira","Mortágua","Nelas","Oliveira de Frades","Penalva do Castelo",
    "Penedono","Resende","Santa Comba Dão","São João da Pesqueira","São Pedro do Sul",
    "Sátão","Sernancelhe","Tabuaço","Tarouca","Tondela","Vila Nova de Paiva",
    "Viseu","Vouzela",
    "Desconhecido - Continente",
    "Calheta (Madeira)","Câmara de Lobos","Funchal","Machico","Ponta do Sol",
    "Porto Moniz","Ribeira Brava","Santa Cruz","Santana","São Vicente","Porto Santo",
    "Vila do Porto","Lagoa (Açores)","Nordeste","Ponta Delgada","Povoação",
    "Ribeira Grande","Vila Franca do Campo","Angra do Heroismo","Praia da Vitória",
    "Santa Cruz da Graciosa","Calheta (Açores)","Velas","Lajes do Pico","Madalena",
    "São Roque do Pico","Horta","Lajes das Flores","Santa Cruz das Flores","Corvo"
]

# --- OPEN EXCEL ---
app = xw.App(visible=True)
wb = xw.Book(excel_file)
ws = wb.sheets[sheet_name]

all_data = []

for municipio in municipios:
    ws.range(municipio_cell).value = municipio

    # force refresh
    wb.app.calculate()
    sleep(2)  # wait for calculations to complete

    table_values = ws.range(data_range).value

    for row in table_values:
        if not row or row[0] is None:
            continue

        categoria = row[0]
        alta = row[1]
        baixa = row[2]
        autoconsumo = row[3]
        total = row[4]

        all_data.append({
            "Município": municipio,
            "Categoria": categoria,
            "Alta": alta,
            "Baixa": baixa,
            "Autoconsumo": autoconsumo,
            "Total": total
        })

df_final = pd.DataFrame(all_data)
df_final.to_csv(output_csv, index=False)

print(f"CSV export complete: {output_csv}")

CSV export complete: /Users/luizbeethoven/Desktop/municipios_output.csv
